# Save and serialize
Keras model consists of:

- architecture
- weights (the "state of the model")
- optimizer
- losses and metrics

You can save:
- everything at once as a TensorFlow SavedModel (or older Keras H5 format)
- only architecture (doesn't apply to subclassed models)
- only weights (generally used when training the model)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

inputs = tf.keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

### All at once

In [ ]:
# old format
model.save("data/model", save_format='h5')
model.save("data/model.h5")
model.save("data/model.keras")

SavedModel format enables Keras to restore both built-in layers as well as custom objects.

In [ ]:
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# save the entire model as a single file
model.save("data/model")
del model

# Recreate the exact same model purely from the file:
model = tf.keras.models.load_model("data/model")

# The reconstructed model is already compiled and has retained the optimizer state,
# so training can resume:
model.fit(test_input, test_target)


When saving the model and its layers, the SavedModel format stores the class name, call() function, losses, and weights (and the config, if implemented). The call function defines the computation graph of the model/layer.

In the absence of the model/layer config, the call function is used to create a model that exists like the original model which can be trained, evaluated, and used for inference.

Nevertheless, it is always a good practice to define the get_config and from_config methods when writing a custom model or layer class. This allows you to easily update the computation later if needed. 

In [ ]:
class CustomModel(tf.keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        self.hidden_units = hidden_units
        self.dense_layers = [tf.keras.layers.Dense(u) for u in hidden_units]

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x

    def get_config(self):
        return {"hidden_units": self.hidden_units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)


model = CustomModel([16, 16, 10])
# Build the model by calling it
input_arr = tf.random.uniform((1, 5))
outputs = model(input_arr)
model.save("my_model")

# Option 1: Load with the custom_object argument.
loaded_1 = tf.keras.models.load_model(
    "my_model", custom_objects={"CustomModel": CustomModel}
)

# Option 2: Load without the CustomModel class.

# Delete the custom-defined model class to ensure that the loader does not have
# access to it.
del CustomModel

loaded_2 = tf.keras.models.load_model("my_model")
np.testing.assert_allclose(loaded_1(input_arr), outputs)
np.testing.assert_allclose(loaded_2(input_arr), outputs)

print("Original model:", model)
print("Model Loaded with custom objects:", loaded_1)
print("Model loaded without the custom object class:", loaded_2)


he first loaded model is loaded using the config and CustomModel class. The second model is loaded by dynamically creating the model class that acts like the original model.

#### Configuring the SavedModel
New in TensoFlow 2.4 The argument save_traces has been added to model.save, which allows you to toggle SavedModel function tracing. Functions are saved to allow the Keras to re-load custom objects without the original class definitons, so when save_traces=False, all custom objects must have defined get_config/from_config methods. When loading, the custom objects must be passed to the custom_objects argument. save_traces=False reduces the disk space used by the SavedModel and saving time.

### Saving the architecture
Sequential model and Functional API model are explicit graphs of layers: their configuration is always available in a structured form.
1. **get_config() / from_config()**

In [ ]:
# layers
layer = tf.keras.layers.Dense(3, activation="relu")
layer_config = layer.get_config()
new_layer = tf.keras.layers.Dense.from_config(layer_config)

# Sequential
model = tf.keras.Sequential([tf.keras.Input((32,)), layers.Dense(1)])
config = model.get_config()
new_model = tf.keras.Sequential.from_config(config)

# Functional
inputs = tf.keras.Input((32,))
outputs = layers.Dense(1)(inputs)
model = tf.keras.Model(inputs, outputs)
config = model.get_config()
new_model = tf.keras.Model.from_config(config)

2. **model_to_json() / model_from_json()**

Turns the model into a JSON string, which can then be loaded without the original model class. It is also specific to models, it isn't meant for layers.

In [ ]:
model = tf.keras.Sequential([tf.keras.Input((32,)), tf.keras.layers.Dense(1)])
json_config = model.to_json()
new_model = tf.keras.models.model_from_json(json_config)

## Custom objects
### Custom Models and layers
In order to save/load a model with custom-defined layers, or a subclassed model, you should overwrite the get_config and optionally from_config methods. Additionally, you should register the custom object so that Keras is aware of it.

Keras keeps a master list of all built-in layer, model, optimizer, and metric classes, which is used to find the correct class to call from_config. If the class can't be found, then an error is raised (Value Error: Unknown layer). There are a few ways to register custom classes to this list:

1. Setting custom_objects argument in the loading function.
2. tf.keras.utils.custom_object_scope or tf.keras.utils.CustomObjectScope
3. tf.keras.utils.register_keras_serializable


In [ ]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name="var_a")

    def call(self, inputs, training=False):
        if training:
            return inputs * self.var
        else:
            return inputs

    # should return a JSON-serializable dictionary
    def get_config(self):
        return {"a": self.var.numpy()}

    # should return a new layer or model object
    @classmethod
    def from_config(cls, config):
        return cls(**config) # this is actually the default behavior


layer = CustomLayer(5)
layer.var.assign(2)

serialized_layer = tf.keras.layers.serialize(layer)
new_layer = tf.keras.layers.deserialize(
    serialized_layer, custom_objects={"CustomLayer": CustomLayer}
)
# this generates a serialized form of the custom layer:
# {'class_name': 'CustomLayer', 'config': {'a': 2} }

### Custom functions
The function name is sufficient for loading as long as it is registered as a custom object.

In [ ]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({"units": self.units})
        return config


def custom_activation(x):
    return tf.nn.tanh(x) ** 2


# Make a model with the CustomLayer and custom_activation
inputs = tf.keras.Input((32,))
x = CustomLayer(32)(inputs)
outputs = tf.keras.layers.Activation(custom_activation)(x)
model = tf.keras.Model(inputs, outputs)

# Retrieve the config
config = model.get_config()

# At loading time, register the custom objects with a `custom_object_scope`:
custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation}
with tf.keras.utils.custom_object_scope(custom_objects):
    new_model = tf.keras.Model.from_config(config)


### Loading the TensorFlow graph only
It's possible to load the TensorFlow graph generated by the Keras. If you do so, you won't need to provide any custom_objects. 

    model.save("my_model")
    tensorflow_graph = tf.saved_model.load("my_model")
    x = np.random.uniform(size=(4, 32)).astype(np.float32)
    predicted = tensorflow_graph(x).numpy()

Note that the loaded object will not be a Keras model, eg., you won't have access to predict() or fit(). Don't use this method unless you're in a tight spot.

### In-memory model cloning

This is equivalent to getting the config then recreating the model from its config (so it does not preserve compilation information or layer weights values).

    with keras.utils.custom_object_scope(custom_objects):
        new_model = keras.models.clone_model(model)

## Saving only weights

- You only need the model for inference: in this case you won't need to restart training, so you don't need the compilation information or optimizer state.
- You are doing transfer learning: in this case you will be training a new model reusing the state of a prior model, so you don't need the compilation information of the prior model.


In [ ]:
def create_layer():
    layer = tf.keras.layers.Dense(64, activation="relu", name="dense_2")
    layer.build((None, 784))
    return layer

layer_1 = create_layer()
layer_2 = create_layer()

# Copy weights from layer 1 to layer 2
layer_2.set_weights(layer_1.get_weights())

Note that stateless layers do not change the order or number of weights, so models can have compatible architectures even if there are extra/missing stateless layers.

In [ ]:
inputs = tf.keras.Input(shape=(784,), name="digits")
x = tf.keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = tf.keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = tf.keras.layers.Dense(10, name="predictions")(x)
functional_model = tf.keras.Model(
    inputs=inputs, outputs=outputs, name="3_layer_mlp"
)

inputs = tf.keras.Input(shape=(784,), name="digits")
x = tf.keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = tf.keras.layers.Dense(64, activation="relu", name="dense_2")(x)
x = tf.keras.layers.Dropout(0.5)(x)    # does not contain any weights
outputs = tf.keras.layers.Dense(10, name="predictions")(x)
functional_model_with_dropout = tf.keras.Model(
    inputs=inputs, outputs=outputs, name="3_layer_mlp"
)

functional_model_with_dropout.set_weights(functional_model.get_weights())

### Saving weights to disk
#### Chekpoint format
This format saves and restores the weights using object attribute names. Eg. `tf.keras.layers.Dense` layer contains two weights: `dense.kernel` and `dense.bias` --> the resulting checkpoint contains the keys "kernel" and "bias" and their corresponding weight values.

    class CustomLayer(keras.layers.Layer):
        def __init__(self, a):
            self.var = tf.Variable(a, name="var_a")

The variable CustomLayer.var is saved with "var" as part of key, not "var_a".

In [1]:
# TF Checkpoint format
sequential_model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(784,), name="digits"),
        tf.keras.layers.Dense(64, activation="relu", name="dense_1"),
        tf.keras.layers.Dense(64, activation="relu", name="dense_2"),
        tf.keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("ckpt")
load_status = sequential_model.load_weights("ckpt")

# validate that all variable values have been restored from the checkpoint
load_status.assert_consumed()


Essentially, as long as two models have the same architecture, they are able to share the same checkpoint. It is generally recommended to stick to the same API for building models. If you switch between Sequential and Functional, or Functional and subclassed, etc., then always rebuild the pre-trained model and load the pre-trained weights to that model.

The next question is, how can weights be saved and loaded to different models if the model architectures are quite different? The solution is to use tf.train.Checkpoint to save and restore the exact layers/variables.

In [ ]:
# Create a subclassed model that essentially uses functional_model's first
# and last layers.
# First, save the weights of functional_model's first and last dense layers.
first_dense = functional_model.layers[1]
last_dense = functional_model.layers[-1]
ckpt_path = tf.train.Checkpoint(
    dense=first_dense, kernel=last_dense.kernel, bias=last_dense.bias
).save("ckpt")

# Define the subclassed model.
class ContrivedModel(tf.keras.Model):
    def __init__(self):
        super(ContrivedModel, self).__init__()
        self.first_dense = tf.keras.layers.Dense(64)
        self.kernel = self.add_variable("kernel", shape=(64, 10))
        self.bias = self.add_variable("bias", shape=(10,))

    def call(self, inputs):
        x = self.first_dense(inputs)
        return tf.matmul(x, self.kernel) + self.bias


model = ContrivedModel()
# Call model on inputs to create the variables of the dense layer.
_ = model(tf.ones((1, 784)))

# Create a Checkpoint with the same structure as before, and load the weights.
tf.train.Checkpoint(
    dense=model.first_dense, kernel=model.kernel, bias=model.bias
).restore(ckpt_path).assert_consumed()


#### HDF5 format
The HDF5 format contains weights grouped by layer names. The weights are lists ordered by concatenating the list of trainable weights to the list of non-trainable weights (same as layer.weights). Thus, a model can use a hdf5 checkpoint if it has the same layers and trainable statuses as saved in the checkpoint.

When loading pretrained weights from HDF5, it is recommended to load the weights into the original checkpointed model, and then extract the desired weights/layers into a new model.

In [ ]:
sequential_model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(784,), name="digits"),
        layers.Dense(64, activation="relu", name="dense_1"),
        layers.Dense(64, activation="relu", name="dense_2"),
        layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("weights.h5")
#or
sequential_model.save_weights("weights", save_format="h5")

sequential_model.load_weights("weights.h5")